In [ ]:
import pandas as pd
import numpy as np
import gc
from lightgbm import LGBMClassifier
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn import metrics
from scipy.optimize import differential_evolution

In [ ]:
active_promos = pd.read_csv("active_promos.csv")
executed_promos = pd.read_csv("executed_promos.csv")
clients_attr = pd.read_csv("clients_attributes.csv")
sales = pd.read_csv("sales.csv", encoding = "ISO-8859-1")
test = pd.read_csv("test_submission.csv")
print("The dimension from the dataset is: {}".format(active_promos.shape))
print("The dimension from the dataset executed_promois: {}".format(executed_promos.shape))
print("The dimension from the dataset clientes_attr is: {}".format(clients_attr.shape))
print("The dimension from the dataset sales is: {}".format(sales.shape))
print("The dimension from the dataset test is: {}".format(test.shape))

# Making datasets

La idea es de crear funciones que automatizen la creacion de los conjuntos de entrenamiento y de test para luego poder evaluar. 

La función principal que realiza todo esto es: **make_datasets** que tiene como entrada las 4 datasets por separado:
* active_promos_df
* executed_promos_df
* client_attributes_df
* sales_df
* test_df 
* meses_key: meses para obtener nuevas variables de la data set SALES
* CodMes_int: el código de mes para el conjunto de test, por ejemplo 201910.

 y la salida son dos conjuntos de datos: train y test.

In [ ]:
# Utils

def get_id(data: pd.DataFrame) -> pd.DataFrame:
    data["Id"] = data.CodigoDC.astype(str) + data.Cliente.astype(str) + data.Marca.astype(str) + data.Cupo.astype(str)
    data = data.set_index("Id")
    return data

def get_data_frame_target(active_promos: pd.DataFrame, executed_promos: pd.DataFrame) -> pd.DataFrame:
    active_promos = get_id(active_promos)
    executed_promos = get_id(executed_promos)

    executed_promos["target"] = 1
    active_promos = active_promos.join(executed_promos.target).replace(np.nan, 0)
    active_promos = active_promos.reset_index().drop("Id", axis=1)
    return active_promos

def mergue_client_attr_target_df(clients_attr: pd.DataFrame, data_frame_target: pd.DataFrame) -> pd.DataFrame:
    clients_attr = clients_attr.set_index("Cliente")
    data_frame_target = data_frame_target.set_index("Cliente")
    data_frame_target = data_frame_target.join(clients_attr)
    data_frame_target = data_frame_target.reset_index()
    return data_frame_target

def get_sales_df(sales: pd.DataFrame) -> pd.DataFrame:
    sales = sales.dropna()
    to_dummies_sales = ["ClaseEnvase", "SegmentoPrecio"]
    Descuento = pd.get_dummies(sales.Dcto == 0)
    Descuento.columns = ["ConDescuento", "SinDescuento"]
    Nr = pd.get_dummies(sales.Nr < 0)
    Nr.columns = ["NrPositivo", "NrNegativo"]
    ClaseEnvase = pd.get_dummies(sales.ClaseEnvase, prefix = "ClaseEnvase")
    SegmentoPrecio = pd.get_dummies(sales.SegmentoPrecio.astype(int), prefix = "SegmentoPrecio")
    sales = sales.join(Descuento).join(Nr).join(ClaseEnvase).join(SegmentoPrecio)
    sales["MesCod"] = sales.Año.astype(str) + sales.Mes.map("{:02}".format)
    sales["MesCod"] = sales.MesCod.astype(int)
    sales["Cupo"] = sales.Cupo.astype(int)
    return sales.drop(["Año", "Mes"], axis=1).drop(to_dummies_sales, axis=1)

def marca_cupo_sales_df(sales: pd.DataFrame, meses) -> pd.DataFrame:
    sales = get_sales_df(sales)
    df_1 = sales.groupby(["MesCod", "Cliente", "Marca", "Cupo"]).sum().sort_index().astype("int32")
    sales_complementos = []
    for mes in meses.keys():
        res_sum = pd.concat([
            df_1.loc[meses[mes]].groupby(["Cliente", "Marca", "Cupo"]).sum()
        ], axis=1)
        res_sum["MesCod"] = mes
        res_sum = res_sum.reset_index().set_index(["Cliente", "Marca", "Cupo", "MesCod"]).astype("float32").add_suffix('_sum')

        res_min = pd.concat([
            df_1.loc[meses[mes]].groupby(["Cliente", "Marca", "Cupo"]).min()
        ], axis=1)
        res_min["MesCod"] = mes
        res_min = res_min.reset_index().set_index(["Cliente", "Marca", "Cupo", "MesCod"]).astype("float32").add_suffix('_min')

        res_max = pd.concat([
            df_1.loc[meses[mes]].groupby(["Cliente", "Marca", "Cupo"]).max()
        ], axis=1)
        res_max["MesCod"] = mes
        res_max = res_max.reset_index().set_index(["Cliente", "Marca", "Cupo", "MesCod"]).astype("float32").add_suffix('_max')

        sales_complementos.append(res_sum.join(res_min).join(res_max))

    gc.collect()
    sales_complementos = pd.concat(sales_complementos)
    return sales_complementos

def personal_info_sales_df(sales: pd.DataFrame, meses) -> pd.DataFrame:
    sales = get_sales_df(sales)
    df_1 = sales.drop(["Marca", "Cupo"], axis = 1).groupby(["MesCod", "Cliente"]).sum().sort_index().astype("int32")
    sales_complementos = []
    for mes in meses.keys():
        res_sum = pd.concat([
            df_1.loc[meses[mes]].groupby("Cliente").sum()
        ], axis=1)
        res_sum["MesCod"] = mes
        res_sum = res_sum.reset_index().set_index(["Cliente", "MesCod"]).astype("float32").add_suffix('_cliente_sum')

        res_min = pd.concat([
            df_1.loc[meses[mes]].groupby("Cliente").min()
        ], axis=1)
        res_min["MesCod"] = mes
        res_min = res_min.reset_index().set_index(["Cliente", "MesCod"]).astype("float32").add_suffix('_cliente_min')

        res_max = pd.concat([
            df_1.loc[meses[mes]].groupby("Cliente").max()
        ], axis=1)
        res_max["MesCod"] = mes
        res_max = res_max.reset_index().set_index(["Cliente", "MesCod"]).astype("float32").add_suffix('_cliente_max')

        sales_complementos.append(res_sum.join(res_min).join(res_max))

    gc.collect()
    print("contatenando complementos")
    sales_complementos = pd.concat(sales_complementos)
    return sales_complementos


def get_train_df(clients_attr: pd.DataFrame, active_promos: pd.DataFrame, executed_promos: pd.DataFrame
                 , marca_cupo_info_sales: pd.DataFrame, personal_info_sales: pd.DataFrame) -> pd.DataFrame:
    data_frame_target = get_data_frame_target(active_promos, executed_promos)
    train = mergue_client_attr_target_df(clients_attr, data_frame_target)
    train["MesCod"] = pd.DatetimeIndex(train.Fecha_Desde).year.astype(str) + pd.DatetimeIndex(
        train.Fecha_Desde).month.map("{:02}".format).astype(str)
    train["MesCod"] = train.MesCod.astype(int)
    train["Grupo"] = 1
    train.loc[(train.MesCod>=201811)&(train.MesCod<=201901),"Grupo"] = 2
    train.loc[(train.MesCod>=201902)&(train.MesCod<=201904),"Grupo"] = 3
    train.loc[(train.MesCod>=201908)&(train.MesCod<=201909),"Grupo"] = 4
    train = train.set_index(["Cliente", "Marca", "Cupo", "MesCod"]).join(
        marca_cupo_info_sales).reset_index().set_index(["Cliente", "MesCod"]).join(personal_info_sales).reset_index()
    print("Dataset train terminada.")
    return train


def get_test_df(test: pd.DataFrame, clients_attr: pd.DataFrame, marca_cupo_info_sales: pd.DataFrame,
                personal_info_sales: pd.DataFrame, MesCod: int) -> pd.DataFrame:
    test["MesCod"] = MesCod
    test = mergue_client_attr_target_df(clients_attr, test)
    test["Grupo"] = 3
    test = test.set_index(["Cliente", "Marca", "Cupo", "MesCod"]).join(marca_cupo_info_sales).reset_index().set_index(
        ["Cliente", "MesCod"]).join(personal_info_sales).reset_index()
    ano = str(MesCod)[0:4]
    mes = str(MesCod)[4:]
    test["Fecha_Hasta"] = pd.to_datetime(str(mes+"/01/"+ano))
    print("Dataset test terminada.")
    return test

In [ ]:
def make_datasets_initial(active_promos_df, executed_promos_df, client_attributes_df, sales_df, test_df, meses_key, CodMes_int):
    marca_cupo_sales = marca_cupo_sales_df(sales_df, meses_key)
    personal_info_sales = personal_info_sales_df(sales_df, meses_key)
    data_train = get_train_df(client_attributes_df, active_promos_df, executed_promos_df
                              , marca_cupo_sales, personal_info_sales)
    data_test = get_test_df(test_df, client_attributes_df, marca_cupo_sales, personal_info_sales, CodMes_int)
    return data_train, data_test

# Feature engineering

In [ ]:
def get_data_final_features(data_final: pd.DataFrame) -> pd.DataFrame:
    to_dummies_clients_attr = ["Region", "Gerencia", "SubCanal", "Estrato", "Mes"]
    missing_columns = ["Nr_sum", "Nr_cliente_sum"]
    for column_i in missing_columns:
        data_final[column_i+"_not_na"] = (data_final[column_i].isna()== False).astype(int) 
    
    data_final["Mes"] = pd.DatetimeIndex(data_final.FechaAltaCliente).month
    data_final["AnosTranscurridos"] =pd.DatetimeIndex(data_final.Fecha_Hasta).year - pd.DatetimeIndex(data_final.FechaAltaCliente).year
    data_final["MesesTranscurridos"] = ( (pd.DatetimeIndex(data_final.Fecha_Hasta) - pd.DatetimeIndex(data_final.FechaAltaCliente))/ np.timedelta64(1, 'D') -
             (pd.DatetimeIndex(data_final.Fecha_Hasta).day+ pd.DatetimeIndex(data_final.FechaAltaCliente).day) )/30
    data_final.TipoPoblacion = data_final.TipoPoblacion.astype(int) - 1
    
    Region_dummies = pd.get_dummies(data_final.Region,prefix = "Region")
    Gerencia_dummies = pd.get_dummies(data_final.Gerencia,prefix = "Gerencia")
    Canales_dummies = pd.get_dummies(data_final.SubCanal,prefix = "Canales")
    Estrato_dummies = pd.get_dummies(data_final.Estrato,prefix = "Estrato")
    Mes_dummies = pd.get_dummies(data_final.Mes,prefix = "Mes")
    Marca_dummies = pd.get_dummies(data_final.Marca,prefix = "Marca")
    
    data_final = data_final.join(Region_dummies).join(Gerencia_dummies).join(Canales_dummies).join(Estrato_dummies).join(Mes_dummies).join(Marca_dummies).drop(to_dummies_clients_attr,axis = 1)
    
    return data_final


def reduce_by_correlation(train_df: pd.DataFrame, cut_off_0_1) -> pd.DataFrame:
    correlated_features = set()
    non = ["Cliente", 'MesCod', 'Marca', 'Cupo', 'target',"Grupo"]
    train_df = train_df.drop(non,axis=1) 
    correlation_matrix = train_df.dropna().corr()
    for i in range(len(correlation_matrix.columns)):
         for j in range(i):
                if abs(correlation_matrix.iloc[i, j]) > cut_off_0_1:
                    colname = correlation_matrix.columns[i]
                    correlated_features.add(colname)
    correlated_features=list(correlated_features)
    return correlated_features


In [ ]:
def make_datasets_final(train: pd.DataFrame, test: pd.DataFrame, var_cutoff, cor_cutoff) -> pd.DataFrame:
    train = get_data_final_features(train).drop(["FechaAltaCliente","Fecha_Hasta","Fecha_Desde","CodigoDC"] ,axis= 1)
    test = get_data_final_features(test).drop(["FechaAltaCliente","Fecha_Hasta"] ,axis= 1)
    test = test.rename(columns = {'Ejecuto_Promo': 'target'}, inplace = False)
    train = train.loc[:,test.columns] #obtener mismas variables
    
    to_drop_correlation_high = reduce_by_correlation(train, cor_cutoff)
    train = train.drop(to_drop_correlation_high, axis = 1)
    test = test.drop(to_drop_correlation_high, axis = 1)
    
    not_incluide= ["Grupo", "target", "MesCod"]
    train_df_var = train.drop(not_incluide, axis = 1)
    to_drop_var_little = train_df_var.loc[:,train_df_var.var() < var_cutoff].columns.to_numpy()
    train = train.drop(to_drop_var_little, axis = 1)
    test = test.drop(to_drop_var_little, axis = 1)
    return train, test

### To submit

In [ ]:
def get_submission_df(test: pd.DataFrame, test_probs: pd.DataFrame):
    test_submit = test.copy()
    test_submit["Ejecuto_Promo"] = test_probs.loc[test_submit.index]
    submission_df = test_submit[["Cliente", "Marca", "Cupo", "Ejecuto_Promo"]].copy()
    submission_df.to_csv('submission.csv',index=None,encoding='utf-8')

<h1><center><font size="6">Datasets creation</font></center></h1>

In [ ]:
meses = {
    201806: slice(201801, 201804),
    201807: slice(201802, 201805),
    201808: slice(201803, 201806),
    201809: slice(201804, 201807),
    201810: slice(201805, 201808),
    201811: slice(201806, 201809),
    201812: slice(201807, 201810),
    201901: slice(201808, 201811),
    201902: slice(201809, 201812),
    201903: slice(201810, 201901),
    201904: slice(201811, 201902),
    201905: slice(201812, 201903),
    201906: slice(201901, 201904),
    201907: slice(201902, 201905),
    201908: slice(201903, 201906),
    201909: slice(201904, 201907),
    201910: slice(201905, 201908)
}
CodMes = 201910
train_diego_inicial, test_diego_inicial = make_datasets_initial(active_promos, executed_promos, 
                            clients_attr, sales, test, meses, CodMes)

In [ ]:
var_cutoff = 0
train_diego, test_diego = make_datasets_final(train_diego_inicial, test_diego_inicial, var_cutoff, 1)

In [ ]:
meses = {
    201806: slice(201801, 201804),
    201807: slice(201801, 201805),
    201808: slice(201801, 201806),
    201809: slice(201801, 201807),
    201810: slice(201801, 201808),
    201811: slice(201801, 201809),
    201812: slice(201801, 201810),
    201901: slice(201801, 201811),
    201902: slice(201801, 201812),
    201903: slice(201801, 201901),
    201904: slice(201801, 201902),
    201905: slice(201801, 201903),
    201906: slice(201801, 201904),
    201907: slice(201801, 201905),
    201908: slice(201801, 201906),
    201909: slice(201801, 201907),
    201910: slice(201801, 201908)
}
CodMes = 201910
train_paolo_inicial, test_paolo_inicial = make_datasets_initial(active_promos, executed_promos, 
                            clients_attr, sales, test, meses, CodMes)

In [ ]:
var_cutoff = 0
train_paolo, test_paolo = make_datasets_final(train_paolo_inicial, test_paolo_inicial, var_cutoff, 1)

# x_train, y_train particition

In [ ]:
y_train_diego = train_diego.reset_index()[["index","target"]].copy().set_index("index")
x_train_diego  = train_diego.drop("target",axis=1)

y_train_paolo = train_paolo.reset_index()[["index","target"]].copy().set_index("index")
x_train_paolo  = train_paolo.drop("target",axis=1)

<h1><center><font size="6">Datasets Modeling using LGBM</font></center></h1>

Presentamos las soluciones despues de optimizar algunos hyperparametros mediante group cross validation.

# LightGBM modelo Diego 

In [ ]:
result = [0.04,7,9,174,5000]
lr = result[0]
max_depth = int(result[1])
lambda_l1 = result[2]
num_itera = int(result[3])
min_data_in_leaf = int(result[4])
group_kfold = GroupKFold(n_splits=4)
train_diego_probsLGBM = []
test_diego_probsLGBM = []

for i,(a,b) in enumerate(group_kfold.split(x_train_diego, y_train_diego.loc[x_train_diego.index], x_train_diego.MesCod)):
    Xt = x_train_diego.loc[a,:].drop("Grupo", axis = 1)
    yt = y_train_diego.loc[Xt.index, "target"]

    Xv = x_train_diego.loc[b,:].drop("Grupo", axis = 1)
    yv = y_train_diego.loc[Xv.index, "target"]

    learner = LGBMClassifier(learning_rate=lr,max_depth=max_depth,num_iterations=num_itera,
                                    lambda_l1=lambda_l1, min_data_in_leaf = min_data_in_leaf,
                            seed = 1997)

    learner.fit(Xt, yt  , eval_metric="auc",eval_set= [(Xt, yt),(Xv, yv)], verbose=174)


    train_diego_probsLGBM.append(pd.Series(learner.predict_proba(Xv)[:, -1],
                                    index=Xv.index, name="probs"+ str(i)))
    test_diego_probsLGBM.append(pd.Series(learner.predict_proba(test_diego.drop(["target","Grupo"],axis=1))[:, -1],
                                    index=test_diego.index, name="probs"+ str(i)))
        
train_diego_probsLGBM = pd.concat(train_diego_probsLGBM, axis=1).mean(axis=1)
test_diego_probsLGBM = pd.concat(test_diego_probsLGBM, axis=1).mean(axis=1)

In [ ]:
result = [0.03,4,7.65,210,820]
lr = result[0]
max_depth = int(result[1])
lambda_l1 = result[2]
num_itera = int(result[3])
min_data_in_leaf = int(result[4])
group_kfold = GroupKFold(n_splits=4)
train_paolo_probsLGBM = []
test_paolo_probsLGBM = []

for i,(a,b) in enumerate(group_kfold.split(x_train_paolo, y_train_paolo.loc[x_train_paolo.index], x_train_paolo.MesCod)):
    Xt = x_train_paolo.loc[a,:].drop("Grupo", axis = 1)
    yt = y_train_paolo.loc[Xt.index, "target"]

    Xv = x_train_paolo.loc[b,:].drop("Grupo", axis = 1)
    yv = y_train_paolo.loc[Xv.index, "target"]

    learner = LGBMClassifier(learning_rate=lr,max_depth=max_depth,num_iterations=num_itera,
                                    lambda_l1=lambda_l1, min_data_in_leaf = min_data_in_leaf,
                            seed = 1997, is_imbalanced = True)

    learner.fit(Xt, yt  , eval_metric="auc",eval_set= [(Xt, yt),(Xv, yv)], verbose=210)


    train_paolo_probsLGBM.append(pd.Series(learner.predict_proba(Xv)[:, -1],
                                    index=Xv.index, name="probs"+ str(i)))
    test_paolo_probsLGBM.append(pd.Series(learner.predict_proba(test_paolo.drop(["target","Grupo"],axis=1))[:, -1],
                                    index=test_paolo.index, name="probs"+ str(i)))
        
train_paolo_probsLGBM = pd.concat(train_paolo_probsLGBM, axis=1).mean(axis=1)
test_paolo_probsLGBM = pd.concat(test_paolo_probsLGBM, axis=1).mean(axis=1)

In [ ]:
AUC_cv_mean = metrics.roc_auc_score(y_train_diego.loc[train_diego_probsLGBM.index,], 
                               np.mean([train_paolo_probsLGBM,train_diego_probsLGBM], axis=0))
AUC_cv_max = metrics.roc_auc_score(y_train_diego.loc[train_diego_probsLGBM.index,], 
                               np.max([train_paolo_probsLGBM,train_diego_probsLGBM], axis=0))
AUC_cv_min = metrics.roc_auc_score(y_train_diego.loc[train_diego_probsLGBM.index,], 
                               np.min([train_paolo_probsLGBM,train_diego_probsLGBM], axis=0))
print("AUC_cv_mean:", AUC_cv_mean)
print("AUC_cv_max:", AUC_cv_max)
print("AUC_cv_min:", AUC_cv_min)

In [ ]:
metrics.roc_auc_score(y_train_diego.loc[train_diego_probsLGBM.index,], 
                                           train_diego_probsLGBM)

In [ ]:
test_probsLGBM = np.min([test_paolo_probsLGBM,test_diego_probsLGBM], axis=0)
test_probsLGBM = pd.Series(test_probsLGBM,index=test_paolo_probsLGBM.index)
get_submission_df(test_diego,test_probsLGBM)